In [59]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [1]:
import sys
sys.path.append('~/aiffel/aiffelthon')

In [ ]:
!ls

In [37]:
!pip install rouge_score
!pip install datasets==1.0.2
# !pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
pip install transformers==4.24.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [39]:
pip install transformer-utils

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [40]:
pip install packaging

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [60]:
# 필요한 라이브러리 불러오기
import datasets
import transformers
import pandas as pd
import re
from datasets import Dataset

#Tokenizer
from transformers import BertTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

#Training
from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
#from seq2seq_training_args import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [ ]:
# 경로 지정
#%cd ~/aiffel/aiffelthon

In [ ]:
#!unzip ~/aiffel/aiffelthon/csv.zip

In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_trainer.py

In [ ]:
#!wget https://github.com/huggingface/transformers/blob/main/src/transformers/utils/import_utils.py

In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/models/encoder_decoder/modeling_encoder_decoder.py

In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/configuration_utils.py

In [61]:
# 데이터 불러오기
train_df = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train_Sum1.csv')
#train_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
train_df.rename(columns = {"input": "Text"}, inplace = True)
train_df.rename(columns = {"sentence_onesent": "Summary"}, inplace = True)
#train_df.rename(columns = {"sentence_per_20": "Summary"}, inplace = True)

val_df = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/val_Sum1.csv')
#val_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
val_df.rename(columns = {"input": "Text"}, inplace = True)
val_df.rename(columns = {"summary_onesent": "Summary"}, inplace = True)
#val_df.rename(columns = {"summary_per_20": "Summary"}, inplace = True)
print(len(train_df), len(val_df))

125690 18300


In [ ]:
train_df.head()

In [ ]:
val_df.head()

In [62]:
# 10번째 row만 추출
train_df = train_df.iloc[range(0, len(train_df), 2)]
val_df = val_df.iloc[range(0, len(val_df), 2)]
print(len(train_df), len(val_df))

62845 9150


In [63]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거
    sentence = re.sub('"','', sentence) # 쌍따옴표 제거
    sentence = re.sub("'",'', sentence) # 따옴표 제거
    sentence = re.sub('\n','', sentence) # \n " 제거
    sentence = re.sub('.{2,3}\W{0,1}기자','', sentence) # 기자 이름 제거
    sentence = re.sub(r'[?.!,][/?.!,]', '', sentence) # 여러개 문장 부호를 하나의 문장부호로 바꿉니다
    sentence = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣a-z0-9]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [64]:
# 전체 Text 데이터에 대한 전처리 (1)
from tqdm import tqdm
clean_text = []

for s in tqdm(train_df['Text']):
    clean_text.append(preprocess_sentence(s))
    
train_df['Text'] = clean_text

100%|██████████| 62845/62845 [00:12<00:00, 4843.48it/s]


In [65]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines = []

for s in tqdm(train_df['Summary']):
      clean_headlines.append(preprocess_sentence(s))
        
train_df['Summary'] = clean_headlines

100%|██████████| 62845/62845 [00:01<00:00, 39742.69it/s]


In [66]:
# 전체 Text 데이터에 대한 전처리 (1)

clean_text_val = []

for s in tqdm(val_df['Text']):
    clean_text_val.append(preprocess_sentence(s))
    
val_df['Text'] = clean_text_val

100%|██████████| 9150/9150 [00:01<00:00, 5006.31it/s]


In [67]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines_val = []

for s in tqdm(val_df['Summary']):
      clean_headlines_val.append(preprocess_sentence(s))
        
val_df['Summary'] = clean_headlines_val

100%|██████████| 9150/9150 [00:00<00:00, 42876.58it/s]


In [17]:
train_df.head()

,Text,Summary
0,내일 주말을 앞두고 있습니다 코로나19에 대한 경각심을 유지하고 생활 속 거리두기를...,코로나19 집단감염의 위험요인인 밀폐되고 밀집한 다중이용시설을 자제하고 생활 속 거...
2,아울러 생활방역 시행일로부터 코로나19 감염위기 상황 종료 시까지 생활방역대책본부를...,생활방역대책본부를 구성하여 추진 상황과 주요 대책을 논의할 계획이고 지자체와 방역당...
4,특별히 교육시설 종교시설 실내 체육시설 그리고 의료기관과 같이 많은 사람들이 이용하...,사람들이 많이 이용하는 시설 종사자는 반드시 진단 검사를 받아야 하며 검사 결과가 ...
6,그러니까 단체방 중에서도 회원가입이라든지 누군가의 허락을 받아서 들어가는 단체방 같...,법 시행령이 통과되면 6개월 이후에 시행되지만 기술적 관리 조치에는 준비가 필요하므...
8,무증상 환자들도 감염력을 가지고 있어서 지역사회의 다양한 공간에서 2차 또는 3차까...,치명률의 세대 간 편차 등의 특성 때문에 전문가들은 백신이나 치료제 개발 전까지 코...


In [18]:
val_df.head()

,Text,Summary
0,새해가 밝았다 또 다른 한 해가 시작되었다 눈이 내린 하얀 설원이 앞에 펼쳐져 있는...,새해라는 미지의 시간에 남길 나의 발자국은 다른 이들에게는 이정표가 될 것이다
2,김정은 국무위원장이 지난달 28일부터 31일까지 열린 노동당 중앙위원회 전원회의 보...,노동당 중앙위원회 전원회의 보고에서 북한의 김 국무위원장이 새로운 전략무기를 목격하...
4,스웨덴에서는 지갑을 몸에 지니지 않아도 일상생활에 불편을 겪지 않는 이들이 있다 엄...,스웨덴에서는 지갑을 몸에 지니지 않아도 엄지와 검지 사이의 손등 표면에 마이크로칩을...
6,핵은 손에 든 채 제재는 해제해 달라는 북한식 셈법은 핵확산금지조약에 정면 배치된다...,북 미 협상에 맥이 빠지자 중국과 러시아는 어느 정도 동력을 유지할 수 있는 6자 ...
8,안철수 전 대표가 그분들을 만날 계획은 현재로썬 전혀 없습니다 2일 안철수 전 국민...,이 인사는 안 전 대표가 황 대표와 유 전 대표와는 만날 계획은 없으며 설 명절 전...


In [68]:
# reset_index 사용
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [20]:
tokenizer = BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")

,Text,Summary
0,새해가 밝았다 또 다른 한 해가 시작되었다 눈이 내린 하얀 설원이 앞에 펼쳐져 있는...,새해라는 미지의 시간에 남길 나의 발자국은 다른 이들에게는 이정표가 될 것이다
2,스웨덴에서는 지갑을 몸에 지니지 않아도 일상생활에 불편을 겪지 않는 이들이 있다 엄...,스웨덴에서는 지갑을 몸에 지니지 않아도 엄지와 검지 사이의 손등 표면에 마이크로칩을...
4,안철수 전 대표가 그분들을 만날 계획은 현재로썬 전혀 없습니다 2일 안철수 전 국민...,이 인사는 안 전 대표가 황 대표와 유 전 대표와는 만날 계획은 없으며 설 명절 전...
6,일본에서 가택연금을 피해 레바논으로 몰래 도주한 카를로스 곤 전 르노 닛산 회장을 ...,전 르노 닛산 회장이 일본 가택연금을 피해 레바논으로 도주했지만 일본과 범죄인 인도...
8,지구를 떠난 소년은 아빠를 찾기 위해 외계 행성을 돌아다닌다 때로는 로봇을 타고 외...,1989년만 하더라도 다들 만화영화 2020년 우주의 원더키디처럼 2020년엔 우주...
...,...,...
9140,글쎄요 저는 문재인 대통령의 국정 수행에 대한 지지도라는 것을 과거에도 그랬지만 엄...,논리적으로 따져보면 말이 안 되기 때문에 심하게 말하면 맹목적인 지지나 무조건 지지다
9142,네 네 디젤이에요 보면 이거 미세먼지 많다고 우리 엄마들 되게 이렇게 하는 부분인데...,타다 드라이버 같은 경우 내일이라도 배차가 없을 경우 바로 해고될 수 있는 비정규직...
9144,기본적으로 지금 정부의 어떤 대응 tf는 국무총리실에서 맡고 있습니다 거기에서 외교...,일본 정부가 약속한 부분이 있지만 기본적으로 정부의 대응 tf는 국무총리실에서 맡고 있다
9146,한편 강화고 최후의 1인은 돌직구 발언으로 에이핑크의 마음을 울렸습니다 이에 에이핑...,성우가 꿈인 형윤이를 응원하기 위해 영국의 인기 드라마 셜록의 셜록 역을 맡은 장 ...


In [70]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
#val_len = len(val_df) // 2
val_data = Dataset.from_pandas(val_df.iloc[::2, :])
test_data=Dataset.from_pandas(val_df.iloc[1::2, :])

In [72]:
print(train_data)
print(val_data)
print(test_data)

Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 62845)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 4575)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 4575)


In [73]:
#tokenizer = BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")
#parameter setting
batch_size=16  #
encoder_max_length=256
decoder_max_length=32

In [24]:
def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]                                               
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    batch["input_ids"] = inputs.input_ids                                                               
    batch["attention_mask"] = inputs.attention_mask                                                     
    batch["decoder_input_ids"] = outputs.input_ids                                                      
    batch["labels"] = outputs.input_ids.copy()                                                          
    # mask loss for padding                                                                             
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]                     
    batch["decoder_attention_mask"] = outputs.attention_mask                                                                              
                                                                                                         
    return batch  

In [ ]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    # because RoBERTa automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [74]:
#processing training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],)

#processing validation data
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],)

  0%|          | 0/3928 [00:00<?, ?ba/s]

  0%|          | 0/286 [00:00<?, ?ba/s]

In [ ]:
type(train_data)

In [26]:
# 인코더 모델 불러오기

bertshared =EncoderDecoderModel.from_pretrained("kykim/bertshared-kor-base")

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [27]:
# set special tokens
#from transformers import EncoderDecoderConfig
bertshared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
bertshared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
bertshared.config.max_length = 32
bertshared.config.early_stopping = True
bertshared.config.no_repeat_ngram_size = 2
bertshared.config.length_penalty = 2.0
bertshared.config.num_beams = 2
bertshared.config.vocab_size = bertshared.config.encoder.vocab_size

In [28]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

In [29]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),}

In [79]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/aiffel/aiffel/aiffelthon/ch/",
    num_train_epochs = 3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
    logging_steps=2000,  # set to 2000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=7500,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    #max_steps=16, # delete for full training,
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True,)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bertshared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 62845
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11784
  Number of trainable parameters = 147298320
/opt/conda/lib/python3.9/site-packages/transformers/models/e

Step,Training Loss
2000,0.000200
4000,0.000900
6000,0.000100
8000,0.000200
10000,0.000000


Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch/checkpoint-500
Configuration saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-500/pytorch_model.bin
Deleting older checkpoint [/aiffel/aiffel/aiffelthon/ch/checkpoint-2500] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch/checkpoint-1000
Configuration saved in /aiffel/aiffel/aiffelthon

TrainOutput(global_step=11784, training_loss=0.00025422160734358645, metrics={'train_runtime': 5243.5999, 'train_samples_per_second': 35.955, 'train_steps_per_second': 2.247, 'total_flos': 3.320076170170368e+16, 'train_loss': 0.00025422160734358645, 'epoch': 3.0})

In [80]:
tokenizer = BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")
model = EncoderDecoderModel.from_pretrained('/aiffel/aiffel/aiffelthon/ch/checkpoint-11500/').to("cuda")
batch_size = 16

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=32, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])
pred_str = results["pred"]
label_str = results["Summary"]

loading file vocab.txt from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/tokenizer_config.json
loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/config.json
Model config EncoderDecoderConfig {
  "_commit_hash": "8ea27677d006a547aee2a7753d1ff18a0346860c",
  "_name_or_path": "kykim/bertshared-kor-base",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaske

  0%|          | 0/286 [00:00<?, ?ba/s]

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for o

In [33]:
print(type(pred_str), type(label_str))

<class 'list'> <class 'list'>


In [58]:
for num in range(10):
    print('predicted sentence : ',pred_str[num])
    print('real sentence : ', label_str[num])
    print('-'*100)

predicted sentence :  일본 사법당국의 출국금지 명령이 내려져있던 카를로스 곤 전 르노닛산 회장이 레바논으로 비밀리에 도주했다.
real sentence :  출국금지 명령을 받은 곤 전 르노닛산 회장이 레바논으로 도주하며 일본의 정치적 박해에서 빠져나왔다고 주장했는데 이에 변호인과 수사 및 출입국 당국 모두 당혹스럽다고 했다
----------------------------------------------------------------------------------------------------
predicted sentence :  통영의 하루락국인 우짜는 우동과 짜장을 한 데 한 번에 끝내 끝내는 얼큰한 음식이며 하루락의 음식이라 함은 여수의
real sentence :  통영에서 우연히 개발한 우동과 짜장을 한 데 섞은 음식 우짜는 입맛 당기는 마력이 있는 해장 음식으로 입소문을 타면서 통영 명물로 자리 잡았다
----------------------------------------------------------------------------------------------------
predicted sentence :  문재인 대통령은 권력기관 개혁과 공정사회 개혁이 그 시작이라고 밝히며 권력기관의 개혁 및 공정 사회 개혁의 시작 이라며 강력한 변화 를 만들어내
real sentence :  어떠한 권력기관도 국민 위에 존재할 수 없다는 문 대통령의 발언은 검찰개혁 속도전의 연장선으로 해석된다
----------------------------------------------------------------------------------------------------
predicted sentence :  한국인의 단백질 보충과 한국인 특유의 한국인 단백질 도저히 조화는 잘 어울리지 않을 것 같은 김치와 김밥, 진차한 김치찌개, 김밥에 라면에 넣어
real sentence :  김 명예회장이 1980년대 초 우리 식문화

In [78]:
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE L SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid)

ROUGE 1 SCORE:  Score(precision=0.0026099737215403767, recall=0.017843351548269583, fmeasure=0.00394730023847285)
ROUGE 2 SCORE:  Score(precision=0.0001961305440057867, recall=0.0017122040072859745, fmeasure=0.0003364788660428919)
ROUGE L SCORE:  Score(precision=0.002561265101635002, recall=0.017715846994535517, fmeasure=0.003887713962727652)
